In [5]:
%load_ext autoreload
from pathlib import Path
import itertools

import torch

from src.data_loader_cifar import dataloader
from src.measures_norm import calculate_generalization_bounds

import src.model_constructor as constructor
import warnings

warnings.filterwarnings("ignore")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
%autoreload 2


def models_iterator(depths, filters_sizes, optimizers, drops, lrs):
    models_to_train = []
    for depth in depths:
        fs = filters_sizes[str(depth)]
        d = drops[str(depth)]
        configurations = list(itertools.product(fs, optimizers, d, lrs))

        for config in configurations:
            filters_size, optimizer, drop, lr = config

            nr_filters = filters_size[:depth]
            conv_layers = constructor.Conv(nr_conv=depth, nr_filters=nr_filters, maxpool_batchnorm=True)
            fc_size = filters_size[depth:]
            act_fun = ["ReLU"] * depth
            dropouts = drop
            fc_layers = constructor.FC(
                nr_fc=depth,
                fc_size=fc_size,
                act_funs=act_fun,
                dropouts=dropouts,
                in_features=conv_layers.finaldim,
                num_classes=10,
                batchnorm=True,
            )

            # Create the model using the CNN constructor
            model = constructor.CNN(
                conv_layers=conv_layers, fc_layers=fc_layers, num_classes=10, lr=lr, optim=optimizer
            )

            # Store the model and its parameters in the list
            model_info = {
                "name": f"{model.name}",
                "model": model,
                "params": {"lr": lr, "optimizer": optimizer},
            }

            models_to_train.append(model_info)

    return models_to_train


def accuracy_score(model, train_dataloader, device):
    model.eval()
    correct = total = 0

    with torch.no_grad():
        for X, y in train_dataloader:
            X, y = X.to(device), y.to(device)
            preds = model(X).argmax(dim=1)
            correct += (preds == y).sum().item()
            total += y.size(0)

    return correct / total


depths = [2, 4]
filters_sizes = {
    "2": [[8, 16, 160, 80], [32, 64, 320, 160]],
    "4": [[4, 8, 16, 32, 200, 200, 160, 80], [8, 16, 32, 64, 400, 320, 160, 80]],
}
lrs = [0.01, 0.001, "scheduler"]
drops = {"2": [[0.0, 0.0], [0.5, 0.2]], "4": [[0.0] * 4, [0.5, 0.3, 0.3, 0.2]]}
optimizers = ["adam", "sgd"]


models_to_train = models_iterator(depths, filters_sizes, optimizers, drops, lrs)

In [4]:
train_dataloader, val_dataloader, test_dataloader = dataloader(
    path=Path("data/processed/cifar"), minibatch_size=32
)

In [ ]:
import os

# Path to the directory
directory_path = "models/conv2True_8_16fcTrue_160ReLU00_80ReLU00_lr001adam/"

# Check if the directory exists
if os.path.exists(directory_path):
    if os.path.isdir(directory_path):
        print(f"Contents of the directory '{directory_path}':")
        for item in os.listdir(directory_path):
            print(item)
    else:
        print(f"The path exists but is not a directory: {directory_path}")
else:
    print(f"The directory does not exist: {directory_path}")


The directory does not exist: models/conv2True_8_16fcTrue_160ReLU00_80ReLU00_lr001adam/


In [10]:
from copy import deepcopy

name = models_to_train[0]["name"]
model = models_to_train[0]["model"]

model_path = f"models/{name}"
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f"Device: {device}")

trained_model_0 = deepcopy(models_to_train[0]["model"])
untrained_model_0 = deepcopy(models_to_train[0]["model"])

trained_model_0.load_state_dict(torch.load(model_path + "/trained.pt", map_location=device))
untrained_model_0.load_state_dict(torch.load(model_path + "/untrained.pt", map_location=device))


trained_model_0.to(device)
untrained_model_0.to(device)

measures = calculate_generalization_bounds(
    trained_model_0, untrained_model_0, train_dataloader, val_dataloader, 3, 32, device
)

Device: cuda


FileNotFoundError: [Errno 2] No such file or directory: 'models/conv2True_8_16fcTrue_160ReLU00_80ReLU00_lr001adam/trained.pt'

In [ ]:
conv2True_32_64fcTrue_320ReLU00_160ReLU00_lr0001_adam